# Renewables pull and strategic push: H-DRI-EAF value chain model - labour cost sensitivity analysis

<hr style="border:1px solid gray">

# Importing libraries for calculations and visualizing results

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import statistics

<hr style="border:1px solid gray">

# Input data

## Mass and energy balance input variables

In [2]:
lambda_h=1.5      #[-] stoichiometric ratio of h2 feed to shaft
alpha=0.94          #[94 %] metallisation achievet in the shaft
m9=1              #[kg/s] mass flow of liquid steel
Delta=0.0           #[-] share of scrap in EAF charge
m21=50            #[kg/tLS] massflow of lime into EAF
T_basis=25+273.15    #[K] reference temperature
T_dr= 800+273.15         #[K] operating temperature of shaft
T_ref=273.15+70       #[K] operating temperature of electrolyser
eta_el=0.72         #[72%] efficiency of electrolyzer
eta_hex=0.75        #[75%] efficiency of heat exchanger

eta_heat=0.85       #[85%] efficiency of converting electricity to heat 

"""The DRI shaft furnace requires additional electricity for the operation
    of the DRI (operating the auxillaiary equipment etc.)
    Reference : 
    http://www.energiron.com/wp-content/uploads/2019/05/
    2008-Environmental-Emissions-Compliance-And-Reduction-Of-Greenhouse-Gases-In-A-DR-EAF-Steel-Plant-2.pdf"""
shaft_aux= 0.08  #[MWh/tLS] corresponding to 80kWh/tLS
P_red_aux = shaft_aux*3600000000/1000   #[J/kgLS]
E_briq = 0.01*3600000   #[J/kg HBI]

___

## Stoichiometric and energy capacity input variables

In [3]:
#### Molecular masses [kg/mol]
# mm = molecular mass 
mm_H2=0.002         #hydrogen molecule
mm_O=0.016          #oxygen
mm_Fe=0.056         #iron
mm_H2O=0.018        #water molecule
mm_Fe3O4=0.232      # iron oxide black
mm_Fe2O3=0.16       # iron oxide hematite
mm_FeO=0.072        # iron oxide wuestite

#### Iron to oxygen ratio for hematite and wuestite [-]
xi_hem=2/3
xi_wue=1

#### Stoichiometric factors of electrolysis reaction H2O = H2 + O [-]
ny_H2O_el=1
ny_H2_el=1
ny_O_el=1
dhr_el=242000  #[J/mol] formation enthalpy of water


#### Stoichiometric factors and enthaplies of hematite reduction Fe2O3 + 3H2 = 2Fe + 3H2O [-]
ny_Fe2O3_hem=1
ny_H2_hem=3
ny_Fe_hem=2
ny_H2O_hem=3
dHb_Fe2O3=-825500   #[J/mol] formation enthalpy of hematite
dHb_w= -242000      #[J/mol] formation enthalpy of water (gaseous)
dHr_hem = dHb_w*ny_H2O_hem-dHb_Fe2O3     #[J/mol] reaction enthalpy


#### Stoichiometric factors of hematite reduction to wuestite [-]
ny_Fe2O3_hemwue=1
ny_H2_hemwue=1
ny_FeO_hemwue=2
ny_H2O_hemwue=1

#### Stoichiometric factors of vritual reaction in EAF: Fe3O4 = 3Fe + 4O4 [-]
ny_Fe3O4_eaf=1
ny_Fe_EAF=3
ny_O_EAF=4


#### Other
hvap_w=2257000   #[J/kg] water enthalpy of evaporation
R_H2=4124        #[J/(kg*K)] hydrogen gas constant

___

## Cost calculations input variables

In [4]:
#Resources
ore_price_USD=140   #[USD/iron ore pellet]
flux_price=90 #[Euro/t] price fluxes of EAF  
alloy_price=1777 #[Euro/t alloys] cost of alloys
electrode_price=4000 #[euro/t electrodes] cost of graphite electrodes 
alloy_cons=11 #[kg/tLS] consumption of alloys
electrode_cons=2 #[kg/tLS] consumption of graphite electrodes
o2_price=60.8 #[Euro/t] Price oxygen can be sold          
o2_sellable=60 #[%] share of oxygen possible to sell        
scrap_price=180 #[Euro/t] price of scrap  

#Operations and maintenance
MO_cost=3 #[3% of CAPEX] maintenance and operation cost
lang_factor = 1  #minimum is 1      
overhead = 0.25
              
#CAPEX
capex_eaf=184 #[Euro/t capacity] EAF CAPEX
capex_dri=230 #[Euro/t capacity] DR shaft CAPEX
EL_capex=0.59 #[Euro/W] electroclyser CAPEX

lifetime=20 #[y] lifetime of eaf and shaft
lifetime_el=10 #[y] electrolyser lifetime

#WACC steel
steel_interest_OECD=0.07 #[5+2%]
steel_interest_nonOECD=0.10 #[8+2%]

#WACC renewable energy
RE_interest_OECD=0.04 #[4%]
RE_interest_nonOECD=0.05 #[5%]


# Exchange rates
# <https://data.ecb.europa.eu/data/datasets/EXR/EXR.A.USD.EUR.SP00.A?chart_props=W3sibm9kZUlkIjoiMzIwNzEyIiwicHJvcGVydGllcyI6W3siY29sb3JIZXgiOiIiLCJjb2xvclR5cGUiOiIiLCJjaGFydFR5cGUiOiJsaW5lY2hhcnQiLCJsaW5lU3R5bGUiOiJTb2xpZCIsImxpbmVXaWR0aCI6IjEuNSIsImF4aXNQb3NpdGlvbiI6ImxlZnQiLCJvYnNlcnZhdGlvblZhbHVlIjpmYWxzZSwiZGF0ZXMiOlsiMjAxOS0xMi0zMVQyMzowMDowMC4wMDBaIiwiMjAyMC0xMi0zMFQyMzowMDowMC4wMDBaIl0sImlzVGRhdGEiOmZhbHNlLCJtb2RpZmllZFVuaXRUeXBlIjoiIiwieWVhciI6ImRhdGV3aXNlIiwic3RhcnREYXRlIjoiMjAyMC0wMS0wMSIsImVuZERhdGUiOiIyMDIwLTEyLTMxIiwic2V0RGF0ZSI6dHJ1ZSwic2hvd1RhYmxlRGF0YSI6ZmFsc2UsImNoYW5nZU1vZGUiOmZhbHNlLCJzaG93TWVudVN0eWxlQ2hhcnQiOmZhbHNlLCJkaXNwbGF5TW9iaWxlQ2hhcnQiOnRydWUsInNjcmVlblNpemUiOiJtYXgiLCJzY3JlZW5XaWR0aCI6MTI4MCwic2hvd1RkYXRhIjpmYWxzZSwidHJhbnNmb3JtZWRGcmVxdWVuY3kiOiJub25lIiwidHJhbnNmb3JtZWRVbml0Ijoibm9uZSIsImZyZXF1ZW5jeSI6Im5vbmUiLCJ1bml0Ijoibm9uZSIsIm1vZGlmaWVkIjoiZmFsc2UiLCJzZXJpZXNLZXkiOiJhbm51YWwiLCJzaG93dGFibGVTdGF0ZUJlZm9yZU1heFNjcmVlbiI6ZmFsc2UsImlzZGF0YWNvbXBhcmlzb24iOmZhbHNlLCJzZXJpZXNGcmVxdWVuY3kiOiJhbm51YWwiLCJpbnRpYWxTZXJpZXNGcmVxdWVuY3kiOiJhbm51YWwiLCJtZXRhZGF0YURlY2ltYWwiOiI0IiwiaXNUYWJsZVNvcnRlZCI6ZmFsc2UsImlzWWVhcmx5VGRhdGEiOmZhbHNlLCJyZXNwb25zZURhdGFFbmREYXRlIjoiMjAyMi0xMi0zMSIsImlzaW5pdGlhbENoYXJ0RGF0YSI6dHJ1ZSwiaXNEYXRlc0Zyb21EYXRlUGlja2VyIjp0cnVlLCJkYXRlUGlja2VyRW5kRGF0ZSI6IjIwMjAtMTItMzEiLCJpc0RhdGVQaWNrZXJFbmREYXRlIjp0cnVlLCJzZXJpZXNrZXlTZXQiOiIiLCJkYXRhc2V0SWQiOiIxOCIsImlzQ2FsbGJhY2siOmZhbHNlLCJpc1NsaWRlclRkYXRhIjp0cnVlLCJpc1NsaWRlckRhdGEiOnRydWUsImlzSW5pdGlhbENoYXJ0RGF0YUZyb21HcmFwaCI6dHJ1ZSwiY2hhcnRTZXJpZXNLZXkiOiJFWFIuQS5VU0QuRVVSLlNQMDAuQSIsInR5cGVPZiI6IiJ9XX1d>
EUR_per_USD_2020=0.8755
EUR_per_USD_2014=0.8237
EUR_per_USD_2016=0.8463
EUR_per_USD_2018=0.8835

# Inflation adjustments 
# <https://data.ecb.europa.eu/data/datasets/ICP/ICP.M.U2.N.000000.4.INX>

##Average HICP Indices
HICP_2020=105.06
HICP_2014=99.81
HICP_2016=100.23
HICP_2018=103.56

##Change factors
HICP_2014_2020=HICP_2020/HICP_2014
HICP_2016_2020=HICP_2020/HICP_2016
HICP_2018_2020=HICP_2020/HICP_2018
HICP_2020_2020=HICP_2020/HICP_2020


### Labour cost calculations input variables

In [5]:
# Wokring hours per year
working_year = 2080  #[hours]

# Labour intensities
H2_labour_int = 2   # [hours/kW installed electrolysers]
Shaft_labour_int = 0.18  #[hours/tDRI]
EAF_labour_int = 0.49159  #[hours/tLS]

In [6]:
#function to convert USD in year x to Euros in year 2020
def conversion_euro (EUR_per_USD, change_factor,value_USD):
    converted_to_euro=(value_USD*EUR_per_USD*change_factor) # [Euro in year 2020]
    return converted_to_euro

In [7]:
#function to calculate steelwage ratios
def steel_wage_ratio (steel_wage, employer_contributions, employees, GNI):
    steel_wage_ratio = ((steel_wage*(1+employer_contributions))/employees)/GNI
    return steel_wage_ratio

In [8]:
#function to average wage to steel workers hourly wage
def steel_wage_h(GNI_2020,steel_wage_ratio,overhead):
    steel_wage_hourly=((GNI_2020*steel_wage_ratio)/working_year)*(1+overhead)  #[Euro/h]
    return steel_wage_hourly

### Country specific cost calculations input variables

In [9]:
# Australia
AUS_steel_wage_USD = 1066007838.5  # [USD/year] 2020 data UNIDO
AUS_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, AUS_steel_wage_USD) # [EUR/year]
AUS_steel_employees = 17876  #[number of employees] 2020 UNIDO
AUS_GNI_2020_USD =53630  # [USD/year/person]
AUS_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, AUS_GNI_2020_USD) # [EUR/year/person]
AUS_employer_contributions = 0.195   #[19.5%] National healthcare: 2%, Pension: 12%, Payroll: 5.5%

#AUS_steel_wage_ratio = steel_wage_ratio(AUS_steel_wage, AUS_employer_contributions, AUS_steel_employees, AUS_GNI_2020)
AUS_steel_wage_ratio=1.3
AUS_status="OECD"
AUS_PiT= 0.45  #[45% personal income tax]

# Brazil
BRA_steel_wage_USD = 1668403172.81  # [USD/year] 2020 data UNIDO
BRA_steel_wage= conversion_euro(EUR_per_USD_2020, HICP_2020_2020, BRA_steel_wage_USD) # [EUR/year]
BRA_steel_employees = 132036  #[number of employees] 2020 UNIDO
BRA_GNI_2020_USD =7910  # [USD/year/person] 
BRA_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, BRA_GNI_2020_USD) # [EUR/year/person]
BRA_employer_contributions = 0.305  # [30.5%] Social contributions: 22.5%, Pension: 8%

#BRA_steel_wage_ratio = steel_wage_ratio(BRA_steel_wage, BRA_employer_contributions, BRA_steel_employees, BRA_GNI_2020)
BRA_steel_wage_ratio = 1.3
BRA_status="non_OECD"
BRA_PiT= 0.28  #[28% personal income tax]

# Chile
CHL_steel_wage_USD = 105310000.25  # [USD/year] 2016 data (latest available) UNIDO
CHL_steel_wage = conversion_euro(EUR_per_USD_2016, HICP_2016_2020, CHL_steel_wage_USD) # [EUR/year]
CHL_steel_employees = 5947.75 #[number of employees] 2016 (latest available data) UNIDO
CHL_GNI_2016_USD = 13440 # [USD/year/person]
CHL_GNI_2016 = conversion_euro(EUR_per_USD_2016, HICP_2016_2020, CHL_GNI_2016_USD) # [EUR/year/person]
CHL_GNI_2020_USD = 13020  # [USD/year/person]
CHL_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, CHL_GNI_2020_USD) # [EUR/year/person]
CHL_employer_contributions = 0.024   #[2.4%]  Social security contributions: 2.4%

#CHL_steel_wage_ratio = steel_wage_ratio(CHL_steel_wage, CHL_employer_contributions, CHL_steel_employees, CHL_GNI_2016)
CHL_steel_wage_ratio = 1.3
CHL_status="OECD"
CHL_PiT= 0.40  #[40% personal income tax]

# China
CHN_steel_wage_USD = 22607804827 # [USD/year] 2016 data (latest available) UNIDO
CHN_steel_wage = conversion_euro(EUR_per_USD_2016, HICP_2016_2020, CHN_steel_wage_USD) # [EUR/year]
CHN_steel_employees = 2733631 #[number of employees] 2016 (latest available data) UNIDO
CHN_GNI_2016_USD = 8210 # [USD/year/person]
CHN_GNI_2016 = conversion_euro(EUR_per_USD_2016, HICP_2016_2020, CHN_GNI_2016_USD) # [EUR/year/person]
CHN_GNI_2020_USD = 10520 #[USD/year/person]
CHN_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, CHN_GNI_2020_USD) # [EUR/year/person]
CHN_employer_contributions = 0.282   #[28.2%]  Social security contributions: 28.2%

#CHN_steel_wage_ratio = steel_wage_ratio(CHN_steel_wage, CHN_employer_contributions, CHN_steel_employees, CHN_GNI_2016)
CHN_steel_wage_ratio = 1.3
CHN_status="non_OECD"
CHN_PiT= 0.45  #[45% personal income tax]

# Japan
JPN_steel_wage_USD = 12496320725 # [USD/year] 2014 data (latest available) UNIDO
JPN_steel_wage = conversion_euro(EUR_per_USD_2014, HICP_2014_2020, JPN_steel_wage_USD) # [EUR/year]
JPN_steel_employees = 294603 #[number of employees] 2014 (latest available data) UNIDO
JPN_GNI_2014_USD = 44440 # [USD/year/person]
JPN_GNI_2014 = conversion_euro(EUR_per_USD_2014, HICP_2014_2020, JPN_GNI_2014_USD) # [EUR/year/person]
JPN_GNI_2020_USD = 40870   #[USD/year/person]
JPN_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, JPN_GNI_2020_USD) # [EUR/year/person]
JPN_employer_contributions = 0.1509  #[15.09%] Social contributions: 15.09%

#JPN_steel_wage_ratio = steel_wage_ratio(JPN_steel_wage, JPN_employer_contributions, JPN_steel_employees, JPN_GNI_2014)
JPN_steel_wage_ratio = 1.3
JPN_status="OECD"
JPN_PiT= 0.47  #[47% personal income tax]

# Norway
NOR_steel_wage_USD = 125040815.8  # [USD/year] 2020 data UNIDO
NOR_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, NOR_steel_wage_USD) # [EUR/year]
NOR_steel_employees = 1919  #[number of employees] 2020 UNIDO
NOR_GNI_2020_USD = 78610  # [USD/year/person]
NOR_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, NOR_GNI_2020_USD) # [EUR/year/person]
NOR_employer_contributions = 0.141  #[14.1%] Employer contribution: 14.1%

#NOR_steel_wage_ratio = steel_wage_ratio(NOR_steel_wage, NOR_employer_contributions, NOR_steel_employees, NOR_GNI_2020)
NOR_steel_wage_ratio =1.3
NOR_status="OECD"
NOR_PiT= 0.47  #[47% personal income tax]

# Germany
GER_steel_wage_USD = 7458201314  # [USD/year] 2020 data UNIDO
GER_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, GER_steel_wage_USD) # [EUR/year]
GER_steel_employees = 125387  #[number of employees] 2020 UNIDO
GER_GNI_2020_USD = 47970  # [USD/year/person]
GER_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, GER_GNI_2020_USD) # [EUR/year/person]
GER_employer_contributions = 0.2115  #[21.15%] Pension: 9.3%, Unemployment: 1.23%, Health: 7.3%, Nursing: 1.7%, Long-term care: 1.525%, Insolvency:0.09% 

#GER_steel_wage_ratio = steel_wage_ratio(GER_steel_wage, GER_employer_contributions, GER_steel_employees, GER_GNI_2020)
GER_steel_wage_ratio = 1.3
GER_status="OECD"
GER_PiT= 0.42  #[42% personal income tax]

# Saudi Arabia
SAU_steel_wage_USD = 741430666.7 # [USD/year] 2018 data (latest available) UNIDO
SAU_steel_wage = conversion_euro(EUR_per_USD_2018, HICP_2018_2020, SAU_steel_wage_USD) # [EUR/year]
SAU_steel_employees = 26818 #[number of employees] 2018 (latest available data) UNIDO
SAU_GNI_2018_USD = 21880 # [USD/year/person]
SAU_GNI_2018 = conversion_euro(EUR_per_USD_2018, HICP_2018_2020, SAU_GNI_2018_USD) # [EUR/year/person]
SAU_GNI_2020_USD = 22430  #[USD/year/person]
SAU_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, SAU_GNI_2020_USD) # [EUR/year/person]
SAU_employer_contributions = 0.1175  #[11.75%] Social insurance tax: 11.75%

#SAU_steel_wage_ratio = steel_wage_ratio(SAU_steel_wage, SAU_employer_contributions, SAU_steel_employees, SAU_GNI_2018)
SAU_steel_wage_ratio = 1.3
SAU_status="non_OECD"
SAU_PiT= 0.0  #[0% personal income tax]

# Sweden
SWE_steel_wage_USD = 1273046522  # [USD/year] 2020 data UNIDO
SWE_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, SWE_steel_wage_USD) # [EUR/year]
SWE_steel_employees = 24461  #[number of employees] 2020 UNIDO
SWE_GNI_2020_USD = 54820  # [USD/year/person]
SWE_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, SWE_GNI_2020_USD) # [EUR/year/person]
SWE_employer_contributions = 0.3142  #[31.42%] Social fees: 31.42%

#SWE_steel_wage_ratio = steel_wage_ratio(SWE_steel_wage, SWE_employer_contributions, SWE_steel_employees, SWE_GNI_2020)
SWE_steel_wage_ratio = 1.3
SWE_status="OECD"
SWE_PiT= 0.52  #[52% personal income tax]

# Turkey
TUR_steel_wage_USD = 1288736628 # [USD/year] 2020 data UNIDO
TUR_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, TUR_steel_wage_USD) # [EUR/year]
TUR_steel_employees = 85089  #[number of employees] 2020 UNIDO
TUR_GNI_2020_USD = 9160  # [USD/year/person]
TUR_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, TUR_GNI_2020_USD) # [EUR/year/person]
TUR_employer_contributions = 0.225  #[22.5%] Social security premiums: 22.5%

#TUR_steel_wage_ratio = steel_wage_ratio(TUR_steel_wage, TUR_employer_contributions, TUR_steel_employees, TUR_GNI_2020)
TUR_steel_wage_ratio = 1.3
TUR_status="OECD"
TUR_PiT= 0.40  #[40% personal income tax]

# Vietnam
VNM_steel_wage_USD = 479061946.6 # [USD/year] 2020 data UNIDO
VNM_steel_wage = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, VNM_steel_wage_USD ) # [EUR/year]
VNM_steel_employees = 84571 #[number of employees] 2020 UNIDO
VNM_GNI_2020_USD = 3450  # [USD/year/person]
VNM_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, VNM_GNI_2020_USD) # [EUR/year/person]
VNM_employer_contributions = 0.215  #[21.5%] Social insurance contribution: 17.5%, Health insurance: 3%, Unemployment insurance: 1%

#VNM_steel_wage_ratio = steel_wage_ratio(VNM_steel_wage, VNM_employer_contributions, VNM_steel_employees, VNM_GNI_2020)
VNM_steel_wage_ratio = 1.3
VNM_status="non_OECD"
VNM_PiT= 0.35  #[35% personal income tax]

# South Africa
ZAF_GNI_2020_USD = 6110   # [USD/year/person]
ZAF_GNI_2020 = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, ZAF_GNI_2020_USD) # [EUR/year/person]
ZAF_employer_contributions= 0.02   #[2%] Skillsdevelopment levy: 1%, Unemployment: 1%

#ZAF_steel_wage_ratio = statistics.mean([AUS_steel_wage_ratio, BRA_steel_wage_ratio, CHL_steel_wage_ratio, CHN_steel_wage_ratio, JPN_steel_wage_ratio, NOR_steel_wage_ratio, GER_steel_wage_ratio, SAU_steel_wage_ratio, SWE_steel_wage_ratio, TUR_steel_wage_ratio, VNM_steel_wage_ratio])
ZAF_steel_wage_ratio = 1.3
ZAF_status="non_OECD"
ZAF_PiT= 0.45  #[45% personal income tax]

## Transport input variables

In [10]:
#### Iron ore and HBI shipping costs
iron_ship_price = 0.55  #[EUR/tonne/day]  
hbi_ship_price = 0.55  #[EUR/tonne/day]  
Iron_ship_speed = 14.5*24    #[nm/day] 
HBI_ship__speed = 14.5*24    #[nm/day]

#### Onshore transport costs - H2 pipelines
# H2 transport by pipelines - based ref: https://www.sciencedirect.com/science/article/pii/S0360319922057603 - see appendix section appendix a
## specific investment cost of onshore pipeline commodities

SIC_USD = (1.3+3.6)/2   # USD/(tH2/year)/km. Mean value between low cost, newly constructed hydrogen pipelines and high cost newly constructed hydrogen pipelines. 
SIC = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, SIC_USD) # EUR/(tH2/year)/km.
opex_pipe_USD= 0.0885 #[USD/tH2/km] opex for hydrogen pipelines
opex_pipe = conversion_euro(EUR_per_USD_2020, HICP_2020_2020, opex_pipe_USD) #[EUR/tH2/km]
k = 0.75 #75% utilization rate
a_r = 0.08  # 8% WACC for the annuity factor
a_n= 55 # 55 year depreciation period

annuity_fact_pipe = a_r/(1-((1+a_r)**(-a_n)))
capex_pipe = (SIC*annuity_fact_pipe)/k   # [EUR/tH2/km]

#cost ratio offshore/onshore pipeline cost
offshore_pipeline= 1.96  #. Transport costs through offshore pipelines are estimated as 1.96 times the costs of using onshore pipelines 


___

## Transport calculation functions

In [11]:
#function to convert km to nautical miles
def km_to_nm_conv(km):
    nm=km* 0.539956803
    return nm

#Iron ore transport costs
def Iron_ore_ship_calc (quant, dist, ref_prod): #[Euro/tLS]
    Iron_ore_ship_cost = (iron_ship_price*quant*(km_to_nm_conv(dist)/Iron_ship_speed))/ref_prod   #[Euro/tLS]
    return Iron_ore_ship_cost


def Iron_ore_transport_calc (CASE, quant, dist, ref_prod): #[Euro/tLS]
    if quant == 0:
        return 0
    else:
        return Iron_ore_ship_calc (quant, dist, ref_prod) #[Euro/tLS]

#HBI transport costs
def HBI_ship_calc (quant,dist,ref_prod): #[Euro/tLS]
    HBI_ship_cost = (hbi_ship_price*quant*(km_to_nm_conv(dist)/HBI_ship__speed))/ref_prod   #[Euro/tLS]
    return HBI_ship_cost

#H2 transport costs
def H2_ship_calc (quant,dist, ref_prod): #[Euro/tLS]
    global H2_ship_cost
    global days_at_port
    quant_kg = quant*1000 #move to kg as new version of H2 transport cost is in kg
    
    # calculations for scale of valuechain for port costs
    tpd = (quant_kg/1000)/365                     # H2 per day to calculate storage need - tonne
    carrier_tonnage = 160000*72/1000    # 11520 tonne carryng capacity of carrier from density of LH2
    days_at_port = carrier_tonnage/tpd  # Days at port lwoer if higher througput in terms of tpd [tonne]
    
    H2_liquefaction_cost = 0.53 + 7.7*el_cost_elec # EUR/kgLH2 accirding to SI doc
    H2_carrier_cost = 4.29e-5*dist #EUR/kgLH2 according to SI doc 
    H2_loading_port_cost = 0.024*(days_at_port + 1) #EUR/kgLH2  according to SI doc
    H2_receiving_port_cost = 0.024*1.56*(days_at_port + 1) #EUR/kgLH2 according to SI doc
    #Total shipping cost from total cost/kgLH2 times quantity transproted in kg and then per tLS given ref prod
    H2_shiping_cost = (H2_liquefaction_cost + H2_carrier_cost + H2_loading_port_cost + H2_receiving_port_cost)*quant_kg/ref_prod
    
    if dist == 0:
        H2_shiping_cost = 0

    return H2_shiping_cost

def H2_pipe_calc (quant, dist, ref_prod):#[Euro/tLS]
    H2_pipe_cost =((capex_pipe+opex_pipe)) *dist #EUR/tonne H2  
    H2_piping_cost = H2_pipe_cost* quant/ref_prod   #[Euro/tLS]
    return H2_piping_cost

def H2_off_pipe_calc (quant, dist, ref_prod):#[Euro/tLS]
    H2_offpipe_cost =((capex_pipe+opex_pipe)) *dist*offshore_pipeline #EUR/tonne H2  
    H2_offpiping_cost = H2_offpipe_cost* quant/ref_prod   #[Euro/tLS]
    return H2_offpiping_cost
        
def H2_transport_calc (CASE, quant, dist, ref_prod):#[Euro/tLS]
    if CASE == "3":
        return H2_pipe_calc(quant, dist, ref_prod)#[Euro/tLS]
    elif CASE == "2b":
        return H2_off_pipe_calc (quant, dist, ref_prod)#[Euro/tLS]
    else:
        if quant == 0:
            return 0
        else:
            return H2_ship_calc (quant, dist, ref_prod)#[Euro/tLS]

<hr style="border:0.5px solid gray">

# Case-specific variables

In [12]:
#function to calculate RE change factor as a function of WACC
def RE_WACC_adjustment (status, el_cost):
    if status== "OECD":
        RE_interest = RE_interest_OECD
    elif status == "non_OECD":
        RE_interest = RE_interest_nonOECD
    RE_WACC_adjustment = (7.4651*RE_interest+0.4954)*el_cost
    return RE_WACC_adjustment

In [13]:
#function to set steel WACC
def steel_interest (status):
    if status== "OECD":
        steel_interest = steel_interest_OECD
    elif status == "non_OECD":
        steel_interest = steel_interest_nonOECD
    return steel_interest 

In [14]:
##### Case 1: 

#Transport distances:
transp_iron_ore_dist_C1 = 11136 #[km] shipping
transp_HBI_dist_C1 = 18775.58      #[km] shipping
transp_H2_dist_C1 = 0  #[km]

#Electricity costs: 
el_cost_eaf_C1_base = 0.075  #[Euro/kWh] #China
el_cost_eaf_C1_status = CHN_status
el_cost_eaf_C1 = RE_WACC_adjustment (el_cost_eaf_C1_status, el_cost_eaf_C1_base)

el_cost_elec_C1_base = 0.055 #[Euro/kWh] #Chile
el_cost_elec_C1_status = CHL_status
el_cost_elec_C1 = RE_WACC_adjustment (el_cost_elec_C1_status, el_cost_elec_C1_base)


el_cost_shaft_C1_base = 0.055 #[Euro/kWh] #Chile
el_cost_shaft_C1_status = CHL_status
el_cost_shaft_C1 = RE_WACC_adjustment (el_cost_shaft_C1_status, el_cost_shaft_C1_base)

# Reference plant size
ref_prod_C1 = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C1 = steel_wage_h(CHL_GNI_2020,CHL_steel_wage_ratio, CHL_PiT)  # [Euro/hour] Chile
wage_shaft_C1 = steel_wage_h(CHL_GNI_2020, CHL_steel_wage_ratio, CHL_PiT)  # [Euro/hour] Chile
wage_eaf_C1 = steel_wage_h(CHN_GNI_2020, CHN_steel_wage_ratio, CHN_PiT)  # [Euro/hour] China

# steel interest rates
steel_interest_elec_C1 = steel_interest (CHL_status)
steel_interest_shaft_C1 = steel_interest (CHL_status)
steel_interest_eaf_C1 = steel_interest (CHN_status)

##### Case 2a:

#Transport distances:
transp_iron_ore_dist_C2a = 6400.51   #[km] shipping
transp_HBI_dist_C2a = 0   #[km]
transp_H2_dist_C2a = 6400.51     #[km] shipping

#Electricity costs: 
el_cost_eaf_C2a_base = 0.078  #[Euro/kWh] #Japan
el_cost_eaf_C2a_status = JPN_status
el_cost_eaf_C2a = RE_WACC_adjustment (el_cost_eaf_C2a_status, el_cost_eaf_C2a_base)

el_cost_elec_C2a_base = 0.065 #[Euro/kWh] #Australia
el_cost_elec_C2a_status = AUS_status
el_cost_elec_C2a = RE_WACC_adjustment (el_cost_elec_C2a_status, el_cost_elec_C2a_base)

el_cost_shaft_C2a_base = 0.078  #[Euro/kWh] #Japan
el_cost_shaft_C2a_status = JPN_status
el_cost_shaft_C2a = RE_WACC_adjustment (el_cost_shaft_C2a_status, el_cost_shaft_C2a_base)

# Reference plant size
ref_prod_C2a = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C2a = steel_wage_h(AUS_GNI_2020, AUS_steel_wage_ratio, AUS_PiT)  # [Euro/hour] Australia
wage_shaft_C2a = steel_wage_h(JPN_GNI_2020, JPN_steel_wage_ratio, JPN_PiT)  # [Euro/hour] Japan
wage_eaf_C2a = steel_wage_h(JPN_GNI_2020, JPN_steel_wage_ratio, JPN_PiT)  # [Euro/hour] Japan
 
# steel interest rates
steel_interest_elec_C2a = steel_interest (AUS_status)
steel_interest_shaft_C2a = steel_interest (JPN_status)
steel_interest_eaf_C2a = steel_interest (JPN_status)
    
##### Case 2b:

#Transport distances:
transp_iron_ore_dist_C2b = 7641.35     #[km] shipping
transp_HBI_dist_C2b = 0   #[km]
transp_H2_dist_C2b = 937     #[km] off-shore pipeline

#Electricity costs: 
el_cost_eaf_C2b_base = 0.075  #[Euro/kWh] #Germany
el_cost_eaf_C2b_status = GER_status
el_cost_eaf_C2b = RE_WACC_adjustment (el_cost_eaf_C2b_status, el_cost_eaf_C2b_base)

el_cost_elec_C2b_base = 0.078 #[Euro/kWh] #Norway
el_cost_elec_C2b_status = NOR_status
el_cost_elec_C2b = RE_WACC_adjustment (el_cost_elec_C2b_status, el_cost_elec_C2b_base)

el_cost_shaft_C2b_base = 0.075  #[Euro/kWh] #Germany
el_cost_shaft_C2b_status = GER_status
el_cost_shaft_C2b = RE_WACC_adjustment (el_cost_shaft_C2b_status, el_cost_shaft_C2b_base)

# Reference plant size
ref_prod_C2b = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C2b = steel_wage_h(NOR_GNI_2020, NOR_steel_wage_ratio, NOR_PiT)  # [Euro/hour] Norway
wage_shaft_C2b = steel_wage_h(GER_GNI_2020, GER_steel_wage_ratio, GER_PiT)  # [Euro/hour] Germany
wage_eaf_C2b = steel_wage_h(GER_GNI_2020, GER_steel_wage_ratio, GER_PiT)  # [Euro/hour] Germany

# steel interest rates
steel_interest_elec_C2b = steel_interest (NOR_status)
steel_interest_shaft_C2b = steel_interest (GER_status)
steel_interest_eaf_C2b = steel_interest (GER_status)


##### Case 3:

#Transport distances:
transp_iron_ore_dist_C3 =  0  #[km]
transp_HBI_dist_C3 = 0  #[km]
transp_H2_dist_C3 =2200.00 #[km]   pipelines

#Electricity costs: 
el_cost_eaf_C3_base = 0.093 #[Euro/kWh]  #Turkey
el_cost_eaf_C3_status = TUR_status
el_cost_eaf_C3 = RE_WACC_adjustment (el_cost_eaf_C3_status, el_cost_eaf_C3_base)

el_cost_elec_C3_base = 0.06 #[Euro/kWh]  #Saudi Arabia
el_cost_elec_C3_status = SAU_status
el_cost_elec_C3 = RE_WACC_adjustment (el_cost_elec_C3_status, el_cost_elec_C3_base)

el_cost_shaft_C3_base = 0.093 #[Euro/kWh]  #Turkey
el_cost_shaft_C3_status = TUR_status
el_cost_shaft_C3 = RE_WACC_adjustment (el_cost_shaft_C3_status, el_cost_shaft_C3_base)

# Reference plant size
ref_prod_C3 = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C3 = steel_wage_h(SAU_GNI_2020, SAU_steel_wage_ratio, SAU_PiT)  # [Euro/hour] Saudi Arabia
wage_shaft_C3 = steel_wage_h(TUR_GNI_2020, TUR_steel_wage_ratio, TUR_PiT)  # [Euro/hour] Turkey
wage_eaf_C3 = steel_wage_h(TUR_GNI_2020, TUR_steel_wage_ratio, TUR_PiT)  # [Euro/hour] Turkey

# steel interest rates
steel_interest_elec_C3 = steel_interest (SAU_status)
steel_interest_shaft_C3 = steel_interest (TUR_status)
steel_interest_eaf_C3 = steel_interest (TUR_status)

##### Case 4:

#Transport distances:
transp_iron_ore_dist_C4 = 0  #[km]
transp_HBI_dist_C4 = 4513.32 #[km]   shipping
transp_H2_dist_C4 = 0  #[km]

#Electricity costs: 
el_cost_eaf_C4_base =  0.095 #[Euro/kWh]  # Vietnam
el_cost_eaf_C4_status = VNM_status
el_cost_eaf_C4 = RE_WACC_adjustment (el_cost_eaf_C4_status, el_cost_eaf_C4_base)

el_cost_elec_C4_base =  0.066 #[Euro/kWh] #Australia
el_cost_elec_C4_status = AUS_status
el_cost_elec_C4 = RE_WACC_adjustment (el_cost_elec_C4_status, el_cost_elec_C4_base)

el_cost_shaft_C4_base =  0.066 #[Euro/kWh] #Australia
el_cost_shaft_C4_status = AUS_status
el_cost_shaft_C4 = RE_WACC_adjustment (el_cost_shaft_C4_status, el_cost_shaft_C4_base)

# Reference plant size
ref_prod_C4 = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C4 = steel_wage_h(AUS_GNI_2020, AUS_steel_wage_ratio, AUS_PiT)  # [Euro/hour] Australia
wage_shaft_C4 = steel_wage_h(AUS_GNI_2020, AUS_steel_wage_ratio, AUS_PiT)  # [Euro/hour] Australia
wage_eaf_C4 = steel_wage_h(VNM_GNI_2020, VNM_steel_wage_ratio, VNM_PiT)  # [Euro/hour] Vietnam

# steel interest rates
steel_interest_elec_C4 = steel_interest (AUS_status)
steel_interest_shaft_C4 = steel_interest (AUS_status)
steel_interest_eaf_C4 = steel_interest (VNM_status)

##### Case 5a:
#Transport distances:
transp_iron_ore_dist_C5a = 0 #[km]
transp_HBI_dist_C5a = 0 #[km]
transp_H2_dist_C5a = 0 #[km]


#Electricity costs: 
el_cost_eaf_C5a_base =  0.100 #[Euro/kWh]  #Sweden
el_cost_eaf_C5a_status = SWE_status
el_cost_eaf_C5a = RE_WACC_adjustment (el_cost_eaf_C5a_status, el_cost_eaf_C5a_base)

el_cost_elec_C5a_base = 0.100 #[Euro/kWh]  #Sweden
el_cost_elec_C5a_status = SWE_status
el_cost_elec_C5a = RE_WACC_adjustment (el_cost_elec_C5a_status, el_cost_elec_C5a_base)

el_cost_shaft_C5a_base = 0.100 #[Euro/kWh]  #Sweden
el_cost_shaft_C5a_status = SWE_status
el_cost_shaft_C5a = RE_WACC_adjustment (el_cost_shaft_C5a_status, el_cost_shaft_C5a_base)

# Reference plant size
ref_prod_C5a = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C5a = steel_wage_h(SWE_GNI_2020, SWE_steel_wage_ratio,SWE_PiT)  # [Euro/hour] Sweden
wage_shaft_C5a = steel_wage_h(SWE_GNI_2020, SWE_steel_wage_ratio, SWE_PiT)  # [Euro/hour] Sweden
wage_eaf_C5a = steel_wage_h(SWE_GNI_2020,SWE_steel_wage_ratio, SWE_PiT)  # [Euro/hour] Sweden

# steel interest rates
steel_interest_elec_C5a = steel_interest (SWE_status)
steel_interest_shaft_C5a = steel_interest (SWE_status)
steel_interest_eaf_C5a = steel_interest (SWE_status)

##### Case 5b:
#Transport distances:
transp_iron_ore_dist_C5b = 0 #[km]
transp_HBI_dist_C5b = 0 #[km]
transp_H2_dist_C5b = 0 #[km]


#Electricity costs: 
el_cost_eaf_C5b_base =  0.064 #[Euro/kWh]  #Sweden
el_cost_eaf_C5b = el_cost_eaf_C5b_base
#el_cost_eaf_C5b_status = SWE_status
#el_cost_eaf_C5b = RE_WACC_adjustment (el_cost_eaf_C5b_status, el_cost_eaf_C5b_base)

el_cost_elec_C5b_base = 0.064 #[Euro/kWh]  #Sweden
el_cost_elec_C5b = el_cost_elec_C5b_base
#el_cost_elec_C5b_status = SWE_status
#el_cost_elec_C5b = RE_WACC_adjustment (el_cost_elec_C5b_status, el_cost_elec_C5b_base)

el_cost_shaft_C5b_base = 0.064 #[Euro/kWh]  #Sweden
el_cost_shaft_C5b = el_cost_shaft_C5b_base
#el_cost_shaft_C5b_status = SWE_status
#el_cost_shaft_C5b = RE_WACC_adjustment (el_cost_shaft_C5b_status, el_cost_shaft_C5b_base)

# Reference plant size
ref_prod_C5b = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C5b = steel_wage_h(SWE_GNI_2020, SWE_steel_wage_ratio,SWE_PiT)  # [Euro/hour] Sweden
wage_shaft_C5b = steel_wage_h(SWE_GNI_2020, SWE_steel_wage_ratio, SWE_PiT)  # [Euro/hour] Sweden
wage_eaf_C5b = steel_wage_h(SWE_GNI_2020,SWE_steel_wage_ratio, SWE_PiT)  # [Euro/hour] Sweden

# steel interest rates
steel_interest_elec_C5b = steel_interest (SWE_status)
steel_interest_shaft_C5b = steel_interest (SWE_status)
steel_interest_eaf_C5b = steel_interest (SWE_status)

##### Case 5c:

#Transport distances:
transp_iron_ore_dist_C5c = 0 #[km]
transp_HBI_dist_C5c = 0 #[km]
transp_H2_dist_C5c = 0 #[km]

#Electricity costs: 
el_cost_eaf_C5c_base = 0.057  #[Euro/kWh]  #South Africa
el_cost_eaf_C5c_status = ZAF_status
el_cost_eaf_C5c = RE_WACC_adjustment (el_cost_eaf_C5c_status, el_cost_eaf_C5c_base)

el_cost_elec_C5c_base = 0.057  #[Euro/kWh]  #South Africa
el_cost_elec_C5c_status = ZAF_status
el_cost_elec_C5c = RE_WACC_adjustment (el_cost_elec_C5c_status, el_cost_elec_C5c_base)

el_cost_shaft_C5c_base = 0.057  #[Euro/kWh]  #South Africa
el_cost_shaft_C5c_status = ZAF_status
el_cost_shaft_C5c = RE_WACC_adjustment (el_cost_shaft_C5c_status, el_cost_shaft_C5c_base)

# Reference plant size
ref_prod_C5c = 2.5*10**6  #[tonne/year]

# steel worker wage
wage_elec_C5c = steel_wage_h(ZAF_GNI_2020, ZAF_steel_wage_ratio, ZAF_PiT)  # [Euro/hour] South Africa
wage_shaft_C5c = steel_wage_h(ZAF_GNI_2020, ZAF_steel_wage_ratio, ZAF_PiT)  # [Euro/hour] South Africa
wage_eaf_C5c = steel_wage_h(ZAF_GNI_2020, ZAF_steel_wage_ratio, ZAF_PiT)  # [Euro/hour] South Africa

# steel interest rates
steel_interest_elec_C5c = steel_interest (ZAF_status)
steel_interest_shaft_C5c = steel_interest (ZAF_status)
steel_interest_eaf_C5c = steel_interest (ZAF_status)

<hr style="border:1px solid gray">

# Input: which case

HDRI Input case:
- Case 1: “Resource endowments steer: Brazil, Chile, China”
- Case 2a: "Importing all resources (H2 shipping): Australia, Japan"
- Case 2b: "Importing all resources (H2 pipeline): Norway, Brazil, Germany"
- Case 3: "H2 trade: Saudi Arabia, Türkiye"
- Case 4: "HBI trade: Australia, Vietnam"
- Case 5a: "Integrated H-DRI-EAF: Sweden - high energy costs"
- Case 5b: "Integrated H-DRI-EAF: Sweden - low energy costs"
- Case 5c: "Integrated H-DRI-EAF: South Africa"

In [15]:
case = input("Enter the case we are running: ")

Enter the case we are running: 5c


<hr style="border:1px solid gray">

# Techno-economic modelling

___

## Material balance

Material balance <br>
– the metallisation alpha is defined as the share of iron present as Fe among all Fe atoms
– to solve EAF mass balance the concept of loadings of oxygen and inert substances (indexes ‘o’ and ‘i’) are used with iron streams as point of reference
– both m6 (ore pellets) and m8 (scrap) contain 5% of inert substances 
– The liquid steel product stream m9 is assumed to be pure iron. The carbon flow is neglected in the material balance and later considered in the emissions calculation. 

### EAF (Electric Arc Furnace) 
Conversion of mass fractions to loadings to respect 5% impurity boundary conditions

In [16]:
y_i8=0.05   #mass fraction of inert material in scrap
y_i6=0.05   #mass fraction of inert material in ore pellets (Fe2O3)

x_o7=(1-alpha)*(mm_O/mm_Fe)     #oxygen loading of HBI stream
x_o6=(1/xi_hem)*(mm_O/mm_Fe)    # oxygen loading of ore stream
x_i6=y_i6/(1-y_i6)              #inert component loading of HBI stream
y_o6 = x_o6/(1+x_o6+x_i6)       # converting oxygen loading of ore back to mass fraction
x_i8=y_i8/(1-y_i8)              #inert component loading of scrap stream

# mass balance of EAF
m7_fe=m9/(1+(Delta/(1-Delta)*(1-y_i8))*(1+x_o7+(1+x_o6)*(y_i6/(1-y_i6))))         #iron mass flow of HBI flow
m6_i=m7_fe*(1+x_o6)*y_i6/(1-y_i6)         #inter part of stream m6; all iron in m6 is passed onto m7
m7=m7_fe*(1+x_o7)+m6_i          #HBI flow; all inert material from m6 is passed on to m7
m8=m7*Delta/(1-Delta)           #scrap flow
m17=m21+m6_i+m7_fe*(x_o7) + m8*y_i8         #slag=lime+ore impurities + oxygen removed in EAF

### Shaft (direct reduction) 

In [17]:
m6_fe=m7_fe       #iron part of m6 (pellets)
m6=m6_fe*(1+x_o6)/(1-y_i6)
m5=lambda_h*m6_fe*(mm_H2/mm_Fe)*(ny_H2_hem/ny_Fe_hem)
m10_w=m7_fe*(mm_H2O/mm_Fe)*(alpha*(ny_H2O_hem/ny_Fe_hem)+(1-alpha)*ny_H2O_hemwue/ny_FeO_hemwue)  #water part of m10
m10_h=m5-m7_fe*(mm_H2/mm_Fe)*(alpha*(ny_H2_hem/ny_Fe_hem)+(1-alpha)*(ny_H2_hemwue/ny_FeO_hemwue)) #hydrogen part of m10
m10=m10_w+m10_h 

### HEX (heat exchanger) 

In [18]:
m11=m10_w
m12=m10_h

### Mixing point (m12, m2, m3)

In [19]:
m3=m5
m2=m3-m12

### Electrolyzer

In [20]:
m13=m11/mm_H2O*mm_O
m1=m2+m13-m11

___

### Compiling all the mass flows in a table

#### Creating table

In [21]:
streams=["m2","m3","m4","m5","m6","m7","m8","m9","m10","m11","m12", "m13","m17","m21"]
len(streams)
description=["Mass flow rate of H2 from electrolyzer",
            "Total mass flow rate of H2 into the condenser",
            "Mass flow reate of H2 from the condenser",
            "Mass flow rate of heated H2 into the shaft",
            "Mass flow rate of iron ore pellets into the shaft",
            "Mass flow rate of HBI from the shaft to the EAF",
            "Mass flow rate of scrap into the EAF",
            "Mass flow rate of produced liquid steel",
            "Mass flow rate of water and unused H2 from the shaft",
            "Mass flow rate of water recycled to the electrolysis",
            "Mass flow rate of unused H2 recycled from/to condenser",
            "Mass flow rate of oxygen out from electrolysis",
            "Mass flow rate of slag out from EAF",
            "Mass flow rate of lime into EAF"]

values=[m2, m3, m5, m5, m6, m7, m8, m9, m10, m11, m12, m13, m17, (m21/1000)]

units=np.repeat("kg/s",len(streams))


mass_balance={"Streams": streams,\
              "Description":description,\
              "Values": values,\
              "Units":units}

#### Filling in values

In [22]:
mass_balance_dri_sf=pd.DataFrame(mass_balance)
mass_balance_dri_sf.index=mass_balance_dri_sf['Streams']
#mass_balance_dri_sf=mass_balance_dri_sf.drop('Streams', axis = 1)
mass_balance_dri_sf

,Streams,Description,Values,Units
Streams,,,,
m2,m2,Mass flow rate of H2 from electrolyzer,0.051429,kg/s
m3,m3,Total mass flow rate of H2 into the condenser,0.080357,kg/s
m4,m4,Mass flow reate of H2 from the condenser,0.080357,kg/s
m5,m5,Mass flow rate of heated H2 into the shaft,0.080357,kg/s
m6,m6,Mass flow rate of iron ore pellets into the shaft,1.503759,kg/s
m7,m7,Mass flow rate of HBI from the shaft to the EAF,1.092331,kg/s
m8,m8,Mass flow rate of scrap into the EAF,0.000000,kg/s
m9,m9,Mass flow rate of produced liquid steel,1.000000,kg/s
m10,m10,Mass flow rate of water and unused H2 from the...,0.491786,kg/s


---

### Transport quantities

In [23]:
##### Case 1: 

#Transport quantities:
transp_iron_ore_quant_C1 = m6*ref_prod_C1 #[tonne/year]
transp_HBI_quant_C1 = m7*ref_prod_C1   #[tonne/year]
transp_H2_quant_C1 = 0  #[tonne/year]


##### Case 2a:

#Transport quantities:
transp_iron_ore_quant_C2a = m6*ref_prod_C2a #[tonne/year]
transp_HBI_quant_C2a = 0   #[tonne/year]
transp_H2_quant_C2a =m2*ref_prod_C2a  # [tonne/year]


##### Case 2b:

#Transport quantities:
transp_iron_ore_quant_C2b = m6*ref_prod_C2b #[tonne/year]
transp_HBI_quant_C2b = 0   #[tonne/year]
transp_H2_quant_C2b =m2*ref_prod_C2b  # [tonne/year]


##### Case 3:

#Transport quantities:
transp_iron_ore_quant_C3 = 0 #[tonne/year]
transp_HBI_quant_C3 = 0 #[tonne/year]
transp_H2_quant_C3 = m2*ref_prod_C3 #[tonne/year]


##### Case 4:

#Transport quantities:
transp_iron_ore_quant_C4 = 0 #[tonne/year]
transp_HBI_quant_C4 =m7*ref_prod_C4  #[tonne/year]
transp_H2_quant_C4 =  0 #[tonne/year]


##### Case 5a:

#Transport quantities:
transp_iron_ore_quant_C5a = 0 #[tonne/year]
transp_HBI_quant_C5a = 0 #[tonne/year]
transp_H2_quant_C5a = 0 #[tonne/year]

##### Case 5b:

#Transport quantities:
transp_iron_ore_quant_C5b = 0 #[tonne/year]
transp_HBI_quant_C5b = 0 #[tonne/year]
transp_H2_quant_C5b = 0 #[tonne/year]


##### Case 5c:

#Transport quantities:
transp_iron_ore_quant_C5c = 0 #[tonne/year]
transp_HBI_quant_C5c = 0 #[tonne/year]
transp_H2_quant_C5c = 0 #[tonne/year]


___

## Pre-energy balance, calculating enthaplies

In [24]:
# Hydrogen enthalpy coefficients
### https://webbook.nist.gov/cgi/cbook.cgi?ID=C1333740&Units=SI&Mask=1#Thermo-Gas

def H2_enthalpy_1(T): #T in range 298 - 1000 K
    """Return the enthalpy of hydrogen in kJ/kg"""
    t=T/1000 
    A=33.066718 
    B=-11.363417
    C=11.432816 
    D=-2.772874 
    E=-0.158558 
    F=-9.980797 
    G=172.707974 
    H=0  
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_H2 
    return H_t 
def H2_enthalpy_2(T):#T in range 1000-2500 K 
    """Return the enthalpy of hydrogen in kJ/kg"""
    t=T/1000 
    A=18.563083 
    B=12.257357 
    C=-2.859786  
    D=0.268238 
    E=1.977990 
    F=-1.147438 
    G=156.288133 
    H=0 
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_H2 
    return H_t 

# Water enthalpy coefficients
"""Return the enthalpy of water in kJ/kg"""
### https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Mask=1#Thermo-Gas
def H2O_enthalpy(T):#500-1700 K - gas phase     
    t=T/1000 
    A=30.09200  
    B=6.832514 
    C=6.793435 
    D=-2.534480 
    E=0.082139 
    F=-250.8810 
    G=223.3967 
    H=-241.8264 
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_H2O 
    return H_t 


### https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Mask=2#Thermo-Condensed
def H2O_enthalpy_2(T):#298_1700 K - liquid phase
    """Return the enthalpy of water in kJ/kg"""
    t=T/1000 
    A =-203.6060
    B =1523.290
    C =-3196.413
    D =2474.455
    E =3.855326
    F =-256.5478
    G =-488.7163
    H =-285.8304
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_H2O
    return H_t 


# Iron enthalpy coefficients (Bhaskar uses gamma-iron. I'm changing the coefficients to use alpha-iron to make it more like Valentin's model)
### https://webbook.nist.gov/cgi/cbook.cgi?ID=C7439896&Units=SI&Mask=2#Thermo-Condensed
## Solid phase
def fe_enthalpy(T):#298-700 K
    """Return the enthalpy of alpha-iron in kJ/kg"""
    t=T/1000
    A= 18.42868
    B= 24.64301
    C= -8.913720
    D= 9.664706
    E= -0.012643
    F= -6.573022
    G= 42.51488
    H= 0.000000
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe
    return H_t
def fe_enthalpy_2(T):#700 - 1042K
    """Return the enthalpy of alpha-iron in kJ/kg"""
    t=T/1000
    A= -57767.65
    B= 137919.7
    C= -122773.2
    D= 38682.42
    E= 3993.080
    F= 24078.67
    G= -87364.01
    H= 0.000000
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe
    return H_t
def fe_enthalpy_3(T):#1042 - 1100K
    """Return the enthalpy of alpha-iron in kJ/kg"""
    t=T/1000
    A= -325.8859
    B= 28.92876
    C= 0.000000
    D= 0.000000
    E= 411.9629
    F= 745.8231
    G= 241.8766
    H= 0.000000
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe
    return H_t

def fe_enthalpy_4(T):#1100 - 1809K
    """Return the enthalpy of alpha-iron in kJ/kg"""
    t=T/1000
    A= -776.7387
    B= 919.4005
    C= -383.7184
    D= 57.08148
    E= 242.1369
    F= 697.6234
    G= -558.3674
    H= 0.000000
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe
    return H_t

## liquid phase
def fe_enthalpy_5(T):#1809-3333K
    """Return the enthalpy of alpha-iron in kJ/kg"""
    t=T/1000
    A= 46.02400
    B= -1.88467*10**(-8)
    C= 6.094750*10**(-9)
    D= -6.640301*10**(-10)
    E= -0.8246121*10**(-9)
    F= -10.80543
    G= 72.54094
    H= 12.39052  
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe
    return H_t


# Wuestite (FeO) enthalpy coefficients
### https://webbook.nist.gov/cgi/cbook.cgi?ID=C1345251&Units=SI&Mask=2#Thermo-Condensed
def feo_enthalpy(T): #298-1650 K
    """Return the enthalpy of wuestite in kJ/kg"""
    t=T/1000
    A=45.75120
    B=18.78553
    C=-5.952201
    D=0.852779
    E=-0.081265
    F=-286.7429
    G=110.3120
    H=-272.0441
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_FeO
    return H_t


# Hematite (Fe2O3) enthalpy coefficients
### https://webbook.nist.gov/cgi/cbook.cgi?ID=C1317608&Units=SI&Type=JANAFS&Table=on#JANAFS
## Solid phase

def fe2o3_enthalpy(T): #298 - 950 K
    """Return the enthalpy of hematite in kJ/kg"""
    t=T/1000
    A= 93.43834
    B= 108.3577
    C= -50.86447
    D= 25.58683
    E= -1.611330
    F= -863.2094
    G= 161.0719
    H= -825.5032
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe2O3
    return H_t    

def fe2o3_enthalpy_2(T): # 950 - 1050 K
    """Return the enthalpy of hematite in kJ/kg"""
    t=T/1000
    A= 150.6240
    B= 0.000000
    C= 0.000000
    D= 0.000000
    E= 0.000000
    F= -875.6066
    G= 252.8814
    H= -825.5032
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe2O3
    return H_t    

def fe2o3_enthalpy_3(T): # 1050 - 2500 K
    """Return the enthalpy of hematite in kJ/kg"""
    t=T/1000
    A= 110.9362
    B= 32.04714
    C= -9.192333
    D= 0.901506
    E= 5.433677
    F= -843.1471
    G= 228.3548
    H= -825.5032
    H_t=(A*t +(B*t**2)/2 +(C*t**3)/3 + (D*t**4)/4-(E/t)+F-H)/mm_Fe2O3
    return H_t 

___

### For calculating P_m5 and  P_rec_hex
Enhtaply of hydrogen and water in flow 3, 12 and 10

#### Enhtalpy of hydrogen (h) h10_h

In [25]:
h10_h2_1 = H2_enthalpy_1(T_ref)
h10_h2_2= H2_enthalpy_2(T_dr)

h10_h2=(h10_h2_2-h10_h2_1)*1000   #[J/kg]

#### Calculating enhtalpy of water (w) h10_w

#### Electrolyser integrated with the shaft - This includes the evaporation enthalpy. So this calculation covers the entire enthalpy difference between 70*C liquid water and 800*C steam. 

In [26]:
t_ev=100 #[*C]
t_conv = 273.15 #[Kelvin = Celsius + 273.15]
T_ev=t_ev + t_conv


#Need to divide into two parts for the different phases. Assuming phase change is taking place at 100*C.
h10_w_l= H2O_enthalpy_2(T_ev) - H2O_enthalpy_2(T_ref)
h10_w_s= H2O_enthalpy(T_dr) - H2O_enthalpy(T_ev)

h10_w = ((h10_w_l+h10_w_s)*1000)+hvap_w

#### Electrolyser separated from the shaft - This includes the evaporation enthalpy. So this calculation covers the entire enthalpy difference between 25*C liquid water and 800*C steam.

In [27]:
t_ev=100 #[*C]
t_conv = 273.15 #[K]
T_ev=t_ev + t_conv


#Need to divide into two parts for the different phases. Assuming phase change is taking place at 100*C.
h10_w_l= H2O_enthalpy_2(T_ev) - H2O_enthalpy_2(T_basis)
h10_w_s= H2O_enthalpy(T_dr) - H2O_enthalpy(T_ev)

h10_ws = ((h10_w_l+h10_w_s)*1000)+hvap_w

### For calculating P_m6 - h6 calculation
* Ore pre-heating (m6) from 25C to 800C

In [28]:
#Defining temperatures
t_m6 = 25   #[*C]
t_conv = 273.15 #[Kelvin = Celsius + 273.15]
T_m6=t_m6+t_conv

#The Shomate equation already calculate the enthalpy as: H° − H°298.15. Meaning we only need one calculation
h6=fe2o3_enthalpy_2(T_dr) *1000

### For calculating the heating of HBI
h_m7fe and  h_m7feo from cp integration between T_basis and T_dr

### Calculating h_m7_fe

In [29]:
T_test = 273.15+650
h_m7_fe = fe_enthalpy_3(T_test)*1000

### Calculating h_m7_feo

In [30]:
T_test = 273.15+650
h_m7_feo=  feo_enthalpy(T_test)*1000

___

## Calculating case-speficic parameters

### Pre-heating water

In [31]:
# temp of water before heating
T_w_in1 = 273.15+25   #[K]

#enthalpy of water before heating
H_w_in1 = H2O_enthalpy_2(T_w_in1)   # [kJ/kg]

#temp of water after heating
T_w_in2 = 273.15 + 70   #[K]

#enthalpy of water after heating
H_w_in2 = H2O_enthalpy_2(T_w_in2)  # [kJ/kg]

# energy need to heat up water
P_wa_in = ((H_w_in2 - H_w_in1)*m11*1000)/eta_heat   #[J/s] = [W]
E_wa_in = P_wa_in/m9   #[J/kgLS]

___

## Adjusting parameters based on the case explored

In [32]:
if case == "1":
    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C1   # [km] road freight
    iron_ore_quant = transp_iron_ore_quant_C1  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C1   # [km] shipping
    hbi_quant = transp_HBI_quant_C1  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C1   # [km] none
    H2_quant = transp_H2_quant_C1  #[tonne/year]
    
    # Heating needs
    hc_dri = "CDRI"
    P_w_in = 0   #[W]
    E_w_in = 0   #[J/kgLS]
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C1
    el_cost_elec = el_cost_elec_C1
    el_cost_shaft = el_cost_shaft_C1
    
    #Reference productio
    ref_prod = ref_prod_C1
    
    #Steel worker wage
    wage_elec = wage_elec_C1
    wage_shaft = wage_shaft_C1
    wage_eaf = wage_eaf_C1
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C1
    steel_interest_shaft = steel_interest_shaft_C1
    steel_interest_eaf = steel_interest_eaf_C1
    
elif case == "2a":
    
    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C2a   # [km] shipping
    iron_ore_quant = transp_iron_ore_quant_C2a  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C2a   # [km] none
    hbi_quant = transp_HBI_quant_C2a  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C2a   # [km] shipping
    H2_quant = transp_H2_quant_C2a  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = P_wa_in   #[W]
    E_w_in = E_wa_in   #[J/kgLS]
    h10_w=h10_ws

    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C2a
    el_cost_elec = el_cost_elec_C2a
    el_cost_shaft = el_cost_shaft_C2a
    
    #Reference production
    ref_prod = ref_prod_C2a
    
    #Steel worker wage
    wage_elec = wage_elec_C2a
    wage_shaft = wage_shaft_C2a
    wage_eaf = wage_eaf_C2a 
 
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C2a
    steel_interest_shaft = steel_interest_shaft_C2a
    steel_interest_eaf = steel_interest_eaf_C2a

elif case == "2b":
    
    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C2b   # [km] shipping
    iron_ore_quant = transp_iron_ore_quant_C2b  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C2b   # [km] none
    hbi_quant = transp_HBI_quant_C2b  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C2b   # [km] shipping
    H2_quant = transp_H2_quant_C2b  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = P_wa_in   #[W]
    E_w_in = E_wa_in   #[J/kgLS]
    h10_w=h10_ws

    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C2b
    el_cost_elec = el_cost_elec_C2b
    el_cost_shaft = el_cost_shaft_C2b
    
    #Reference production
    ref_prod = ref_prod_C2b
    
    #Steel worker wage
    wage_elec = wage_elec_C2b
    wage_shaft = wage_shaft_C2b
    wage_eaf = wage_eaf_C2b
    
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C2b
    steel_interest_shaft = steel_interest_shaft_C2b
    steel_interest_eaf = steel_interest_eaf_C2b    
    
    
elif case == "3":
    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C3   # [km] none
    iron_ore_quant = transp_iron_ore_quant_C3  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C3   # [km] none
    hbi_quant = transp_HBI_quant_C3  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C3   # [km] pipelines/roadfreight
    H2_quant = transp_H2_quant_C3  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = P_wa_in   #[W]
    E_w_in = E_wa_in   #[J/kgLS]
    h10_w=h10_ws
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C3
    el_cost_elec = el_cost_elec_C3
    el_cost_shaft = el_cost_shaft_C3
    
    #Reference production
    ref_prod = ref_prod_C3
    
    #Steel worker wage
    wage_elec = wage_elec_C3
    wage_shaft = wage_shaft_C3
    wage_eaf = wage_eaf_C3
    
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C3
    steel_interest_shaft = steel_interest_shaft_C3
    steel_interest_eaf = steel_interest_eaf_C3    
    
    
elif case == "4":
    
    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C4   # [km] none
    iron_ore_quant = transp_iron_ore_quant_C4  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C4   # [km] shipping
    hbi_quant = transp_HBI_quant_C4  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C4   # [km] none
    H2_quant = transp_H2_quant_C4  #[tonne/year]
    
    # Heating needs
    hc_dri = "CDRI"
    P_w_in = 0   #[W]
    E_w_in = 0 #[J/kgLS] 
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C4
    el_cost_elec = el_cost_elec_C4
    el_cost_shaft = el_cost_shaft_C4
    
    #Reference production
    ref_prod = ref_prod_C4  

    #Steel worker wage
    wage_elec = wage_elec_C4
    wage_shaft = wage_shaft_C4
    wage_eaf = wage_eaf_C4

    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C4
    steel_interest_shaft = steel_interest_shaft_C4
    steel_interest_eaf = steel_interest_eaf_C4    
    

elif case == "5a":

    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C5a   # [km] none
    iron_ore_quant = transp_iron_ore_quant_C5a  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C5a   # [km] none
    hbi_quant = transp_HBI_quant_C5a  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C5a   # [km] none
    H2_quant = transp_H2_quant_C5a  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = 0   #[W]
    E_w_in = 0 #[J/kgLS] 
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C5a
    el_cost_elec = el_cost_elec_C5a
    el_cost_shaft = el_cost_shaft_C5a
    
    #Reference production
    ref_prod = ref_prod_C5a
    
    #Steel worker wage
    wage_elec = wage_elec_C5a
    wage_shaft = wage_shaft_C5a
    wage_eaf = wage_eaf_C5a
    
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C5a
    steel_interest_shaft = steel_interest_shaft_C5a
    steel_interest_eaf = steel_interest_eaf_C5a
    
    
elif case == "5b":

    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C5b   # [km] none
    iron_ore_quant = transp_iron_ore_quant_C5b  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C5b   # [km] none
    hbi_quant = transp_HBI_quant_C5b  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C5b   # [km] none
    H2_quant = transp_H2_quant_C5b  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = 0   #[W]
    E_w_in = 0 #[J/kgLS] 
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C5b
    el_cost_elec = el_cost_elec_C5b
    el_cost_shaft = el_cost_shaft_C5b
    
    #Reference production
    ref_prod = ref_prod_C5b
    
    #Steel worker wage
    wage_elec = wage_elec_C5b
    wage_shaft = wage_shaft_C5b
    wage_eaf = wage_eaf_C5b
    
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C5b
    steel_interest_shaft = steel_interest_shaft_C5b
    steel_interest_eaf = steel_interest_eaf_C5b
    
elif case == "5c":

    #Iron ore-transport related inputs
    iron_ore_dist = transp_iron_ore_dist_C5c   # [km] none
    iron_ore_quant = transp_iron_ore_quant_C5c  # [tonne/year] 
    
    #HBI-transport related inputs
    hbi_dist = transp_HBI_dist_C5c   # [km] none
    hbi_quant = transp_HBI_quant_C5c  # [tonne/year]
    
    #H2-transport related inputs
    H2_dist = transp_H2_dist_C5c   # [km] none
    H2_quant = transp_H2_quant_C5c  #[tonne/year]
    
    # Heating needs
    hc_dri = "HDRI"
    P_w_in = 0   #[W]
    E_w_in = 0 #[J/kgLS] 
    
    # Electricity costs
    el_cost_eaf = el_cost_eaf_C5c
    el_cost_elec = el_cost_elec_C5c
    el_cost_shaft = el_cost_shaft_C5c
    
    #Reference production
    ref_prod = ref_prod_C5c
    
    #Steel worker wage
    wage_elec = wage_elec_C5c
    wage_shaft = wage_shaft_C5c
    wage_eaf = wage_eaf_C5c
    
    
    #Steel interest rates
    steel_interest_elec=steel_interest_elec_C5c
    steel_interest_shaft = steel_interest_shaft_C5c
    steel_interest_eaf = steel_interest_eaf_C5c

___

## Energy balance

### P3 - power needed for the EAF

In [33]:
P3_0=2.4*1e06                      #[J/kgLS]
met_plus=11*3.6*1e03               #[J/(kgLS*%met)] adjustment factor for metallisation
dri_plus=0.85*3.6*1e03             #[J/(kgLS*%dir)] adjustment factor for DRI input to EAF
P3=(P3_0+(met_plus*(0.94-alpha)*1e02)+(dri_plus*(1-Delta)*1e02))*m9       #[W] SEC of EAF

E_y_eaf=P3*24*365/1000             #[kWh/year]
E_P3= P3/m9      #Specific energy consumption of the EAF in [J/kgLS]

### P_m5 and P_rec_hex - HEX balance, recoverable heat from then HEX and eventual additional power needed to heat H2 before the shaft

#### m10 enthalpy calculation, recoverable heat in the HEX
- assumption: T_m12=T_m11=70 Celcius for the cases when the electrolysis is integrated with the rest. Otherwise, T_m12=T_m11 = 25*C.
- h10_h and h10_w [J/kg] from cp integration between T_ref and T_dr

In [34]:
HH10_h=h10_h2*m10_h   #[W]
HH10_w=m10_w*(h10_w)      #[W]
HH10=HH10_h+HH10_w    #[W]
HH10_hex=HH10*eta_hex    #[W] recoverable heat in HEX

#### M5 temperature calculation, and calculating P_m5 and P_rec_hex

In [35]:
h5_h2_needa = H2_enthalpy_1(T_ref)
h5_h2_needb = H2_enthalpy_2(T_dr)

h5_h2_need = (h5_h2_needb-h5_h2_needa)*1000

In [36]:
HH5_need=(h5_h2_need*m5)/eta_heat   #[W]
E_H2_preheat_need = HH5_need/m9    #[J/kgLS]
E_rec_heat_tot = HH10_hex/m9    #[J/kgLS]

if HH5_need >= HH10_hex:
    P_m5=(HH5_need - HH10_hex)   #[W]
    P_rec_hex = 0              #[W]
    
elif HH10_hex > HH5_need:
    P_rec_hex = HH10_hex - HH5_need    #[W]
    P_m5= 0                          #[W]    
    
E_Pm5 = P_m5/m9    #[J/kgLS]
E_Prechex = P_rec_hex/m9    #[J/kgLS]

### P_m6 - DRI heat balance, power needed to preheat iron ore before the shaft
- Ore pre-heating (m6) from 25*C to 800*C

In [37]:
P_m6=(h6*m6)/eta_heat   #[W]
E_P6 = P_m6/m9  #[J/kgLS]

### P_red - energy needed to reduce iron in the shaft

In [38]:
reaction_moles=(m6*(1-0.05))/mm_Fe2O3
P_red=reaction_moles*dHr_hem + (P_red_aux*m9)    # [W] Hematite reduction
E_Pred = (P_red/m9) #[J/kgLS]

### P1 - Electrolyzer, power needed for the electroclyzer

In [39]:
P1=(m11*((1/mm_H2O)*dhr_el)/eta_el)+P_w_in      #[W]
E_y_el=P1*24*365/1000    #[kWh] energy needed for yearly operation
EI_el=P1/m9     #[kJ/tLS] or [J/kgLS]

### P_cdri - Heating of HBI  and compression DRI to briquettes
- Energy lost when HBI is cooled down to reference temperature
- if the shaft and EAF are separated, the CDRI needs to be compressed and the reheated to HBI. Both of these processes need energy
- h_m7fe and  h_m7feo from cp integration between T_basis and T_dr

In [40]:
if hc_dri == "CDRI": # [K]
    P_cdri = (((m7_fe*alpha+m6_i)*h_m7_fe+(m7_fe*(1-alpha+x_o7))*h_m7_feo)/eta_heat)+E_briq*m7  #[W]
elif hc_dri == "HDRI": 
    P_cdri = 0
    
E_Pcdri = P_cdri/m9   # #[J/kgLS]

### Total SEC - Specific energy consumption of the steel production process

In [41]:
P_total = P1+P_red + P3 - P_rec_hex + P_m5 + P_m6 + P_cdri   #Total power needed for system [W]
SEC = P_total/m9    #[J/kg] specific energy consumption
e_y_total = P_total*24*365/1000   #[kWh]

___

## Transport cost calculation

In [42]:
# Iron ore
transp_iron_ore_pellets= Iron_ore_transport_calc (case, iron_ore_quant, iron_ore_dist, ref_prod) #[Euro/tLS]

# HBI
transp_HBI = HBI_ship_calc(hbi_quant, hbi_dist, ref_prod) #[Euro/tLS]

# H2
transp_H2 = H2_transport_calc (case, H2_quant, H2_dist, ref_prod) #[Euro/tLS]

# total transport costs
transp_cost = transp_iron_ore_pellets + transp_HBI + transp_H2 #[Euro/tLS]

___

##  Economic calculations

### Pre-cost calculation
Defining electrolyzer capacity for the CAPEX calculations.

The electrolyzer is sized for the case when no scrap is fed into the EAF, which in turn determines the CAPEX of the electrolyzer. If only DRI is fed to EAF then P_mac is written into the P1_D0 parameter. This is needed to determine the CAPEX of the electrolyzer, which is based on the 100% DRI case. The calculation has to be done once with only DRI feed so that the electrolyzer capacity is calculated which is needed for EL CAPEX determination

In [43]:
Delta_0=0

m7_fe_D0=m9/(1+(Delta_0/(1-Delta_0)*(1-y_i8))*(1+x_o7+(1+x_o6)*(y_i6/(1-y_i6))))         #iron mass flow of HBI flow
m10_w_D0=m7_fe_D0*(mm_H2O/mm_Fe)*(alpha*(ny_H2O_hem/ny_Fe_hem)+(1-alpha)*ny_H2O_hemwue/ny_FeO_hemwue)  #water part of m10
m11_D0=m10_w_D0

P1_D0=m11_D0*((1/mm_H2O)*dhr_el)/eta_el      #[W]

### Production capacity

In [44]:
y_prod_mac = ref_prod                   #[t/y] 
hour_prod_mac = y_prod_mac /8760       #[t/h]
sec_prod_mac = hour_prod_mac/3600      #[t/s]

### Defining energy unit conversion function

In [45]:
#Defining a function to converst the units for energy consumption from [J/kgLS] to [MWh/tLS]
def energy_unit_conv(SEC):
    SEC_MWh=SEC/(3.6*1e09)
    SEC_MWh_tLS=format(SEC_MWh*1000, ".5f")
    return SEC_MWh_tLS

### Compiling electricity consumptions

In [46]:
###### Electrolyzer
# Electricity consumption of the electrolyzer
elcons_elec = float(energy_unit_conv(EI_el)) # [MWh/tLS]


##### Shaft
# Electricity consumption of the shaft (the net consumption of electricity to reduce iron ore, auxilliary requirements and the recoverable heat that can be used)
elcons_Shaft= float(energy_unit_conv(E_Pred-E_Prechex))   # [MWh/tLS]

# Electricity consumption to pre-heat iron ore before the shaft
elcons_iron_ore = float(energy_unit_conv(E_P6))    # [MWh/tLS]

# Electricity preheat H2 before the shaft
elcons_H2 = float(energy_unit_conv(E_Pm5))  # [MWh/tLS]


##### EAF
# Electricity consumption the EAF
elcons_EAF = float(energy_unit_conv(E_P3))   # [MWh/tLS]

#Electricity consumption of pre-heating HBI if it has cooled down
elcons_CDRI = float(energy_unit_conv(E_Pcdri))

---

### CAPEX

In [47]:
# Electrolyzer CAPEX
P_mac=(P1_D0/m9)*sec_prod_mac*10**3    #[W] installed electrolyzer capacity for sample steel plant
#P_mac=((P1)/m9)*sec_prod_mac*10**3    #[W] installed electrolyzer capacity for sample steel plant
P_el_capex=P_mac
capex_el=P_el_capex*EL_capex   #[Euro] due to: P_mac [W]*EL_capex [euro/W installed]
CAPEX_el=capex_el/y_prod_mac    #[Euro / t capacity]

#Calculating CAPEX/tonne capacity
tot_CAPEX_per_cap = round((CAPEX_el + capex_eaf + capex_dri)*lang_factor ,2)   #[Euro / t capacity]

#Calculating CAPEX
tot_capex = round((capex_el + ((capex_eaf + capex_dri)*y_prod_mac))*lang_factor/1000000,2)   # [million Euro]

In [48]:
#Calculating the annual capital costs

## Electrolyzer
ACC_el = capex_el * steel_interest_elec / (1 - ((1 + steel_interest_elec) ** (-lifetime_el))) / y_prod_mac    #[€/t capacity]

#EAF
ACC_eaf= capex_eaf * steel_interest_eaf / (1 - ((1 + steel_interest_eaf) ** (-lifetime)))    #[€/t capacity]

#Shaft
ACC_shaft = capex_dri * steel_interest_shaft / (1 - ((1 + steel_interest_shaft) ** (-lifetime)))    #[€/t capacity]

# Total
ACC_tot = ACC_el+ ACC_eaf+ ACC_shaft

#Total 
ACC_total_all_year = capex_el * steel_interest_elec / (1 - ((1 + steel_interest_elec) ** (-lifetime_el))) + capex_eaf *y_prod_mac*steel_interest_eaf / (1 - ((1 + steel_interest_eaf) ** (-lifetime))) +capex_dri * y_prod_mac* steel_interest_shaft / (1 - ((1 + steel_interest_shaft) ** (-lifetime)))  #[€/y total]



### OPEX

In [49]:
# Resource costs
iron_ore_opex = ((ore_price_USD*EUR_per_USD_2020)*(m6/m9))    #[Euro/tLS]
eaf_flux_opex = ((flux_price/1000)*m21)    #[Euro/tLS]
scrap_opex = (scrap_price*(m8/m9))     #[Euro/tLS]
alloy_opex = ((alloy_price/1000)*alloy_cons)    #[Euro/tLS]
graphite_elec_opex = (electrode_cons*(electrode_price/1000))   #[Euro/tLS]

resource_cost=iron_ore_opex+eaf_flux_opex+scrap_opex+alloy_opex+graphite_elec_opex # [Euro/tLS]

#Electricity costs
encost_ton=((elcons_elec)*1000*el_cost_elec)+((elcons_Shaft+elcons_iron_ore+elcons_H2)*1000*el_cost_shaft)+((elcons_EAF+elcons_CDRI)*1000*el_cost_eaf)  #[Euro/tLS] 

#labour cost
labour_cost_elec = wage_elec*H2_labour_int*((P_mac/1000)/y_prod_mac)  #[Euro/tLS]
labour_cost_shaft = wage_shaft*Shaft_labour_int*m7   #[Euro/tLS]
labour_cost_eaf = wage_eaf*EAF_labour_int*m9     #[Euro/tLS]

labour_cost_tot = labour_cost_elec+labour_cost_shaft+labour_cost_eaf

#maintenance costs
maintenance_cost =(MO_cost*10**-2)*(tot_CAPEX_per_cap)  # [Euro/tLS]

#O2 revenues
O2_rev=-((m13/m9)*o2_price*(o2_sellable*10**-2)) #[Euro/tLS] 

# Total OPEX
tot_opex=round(resource_cost+encost_ton+labour_cost_tot+maintenance_cost+O2_rev,2)

<hr style="border:1px solid gray">

# Compile results

## Energy consumption results

In [50]:
description=["Steel plant energy consumption",
            "Electroclyser energy consumption",
            "Electric arc furnace energy consumption",
            "Pre-heating iron ore (before shaft) energy consumption",
            "Heating H2 before shaft energy needed",
            "Iron ore reduction (in shaft) energy needed",
            "Residual energy available from the HEX process, \n after pre-heating H2, that can be further used for heating the shaft",
            "Actual H2 preheat energy consumption",
            "Actual shaft energy consumption",
            "Energy needed to heat the briquetted iron before the EAF"]

values=[energy_unit_conv(SEC),
       energy_unit_conv(EI_el),
       energy_unit_conv(E_P3),
       energy_unit_conv(E_P6),
       energy_unit_conv(E_H2_preheat_need),
       energy_unit_conv(E_Pred),
       energy_unit_conv(-E_rec_heat_tot),
       energy_unit_conv(E_Pm5),
       energy_unit_conv(E_Pred-E_Prechex),
       energy_unit_conv(E_Pcdri)]

units = np.repeat("MWh/tLS", len(description))

Energy_results={"Process step": description,\
               "Values": values,\
               "Units": units}

Energy_res_table=pd.DataFrame(Energy_results)
Energy_res_table.index=Energy_res_table["Process step"]
#Energy_res_table=Energy_res_table.drop("Process step", axis = 1)
Energy_res_table

,Process step,Values,Units
Process step,,,
Steel plant energy consumption,Steel plant energy consumption,3.66842,MWh/tLS
Electroclyser energy consumption,Electroclyser energy consumption,2.40079,MWh/tLS
Electric arc furnace energy consumption,Electric arc furnace energy consumption,0.75167,MWh/tLS
Pre-heating iron ore (before shaft) energy consumption,Pre-heating iron ore (before shaft) energy con...,0.34258,MWh/tLS
Heating H2 before shaft energy needed,Heating H2 before shaft energy needed,0.28356,MWh/tLS
Iron ore reduction (in shaft) energy needed,Iron ore reduction (in shaft) energy needed,0.32678,MWh/tLS
"Residual energy available from the HEX process, \n after pre-heating H2, that can be further used for heating the shaft",Residual energy available from the HEX process...,-0.43695,MWh/tLS
Actual H2 preheat energy consumption,Actual H2 preheat energy consumption,0.00000,MWh/tLS
Actual shaft energy consumption,Actual shaft energy consumption,0.17338,MWh/tLS


## Economic results

### Structuring the economic results

In [51]:
##########################################################################################################
# Electrolyzer
## CAPEX
CAPEX_el  ##[Euro / t capacity]
ACC_el

## Electricity costs
elec_electricity_cost=elcons_elec*el_cost_elec*1000  #[Euro/tLS]

## O&M costs
elec_OM_cost = ((MO_cost*10**-2)*CAPEX_el)  #[Euro/tLS]

## Labour costs
labour_cost_elec   

## O2 revenues
O2_rev

##########################################################################################################

# EAF
## CAPEX
capex_eaf
ACC_eaf

## Resource costs
eaf_resource_costs = eaf_flux_opex + scrap_opex + alloy_opex+graphite_elec_opex

## Electricity costs
eaf_electricity_cost=elcons_EAF*el_cost_eaf*1000

## O&M costs
eaf_OM_cost=((MO_cost*10**-2)*capex_eaf)

## Labour costs
labour_cost_eaf

##########################################################################################################

# Iron ore pre-heat

## CAPEX

## Resource costs
iron_ore_opex = ((ore_price_USD*EUR_per_USD_2020)*(m6/m9))    #[Euro/tLS]

## Electricity costs
iron_ore_preheat_electricity_cost=(elcons_iron_ore*el_cost_shaft*1000)

##########################################################################################################

# H2 pre-heat (actual)

## Electricity costs
H2_preheat_electricity_cost=(elcons_H2*el_cost_shaft*1000)

##########################################################################################################

# Shaft (actual)

## CAPEX
capex_dri
ACC_shaft

## Resource costs

## Electricity costs
shaft_electricity_cost=(elcons_Shaft*el_cost_shaft*1000)

## O&M costs
shaft_OM_cost=((MO_cost*10**-2)*capex_dri)

## Labour costs
labour_cost_shaft

##########################################################################################################

# HBI pre-heat

## CAPEX

## Resource costs

## Electricity costs
HBI_preheat_electricity_cost=(elcons_CDRI*el_cost_eaf*1000)

## O&M costs

## Labour costs

## Graphite electrode

## O2 revenues

##########################################################################################################

# Iron ore transport
## transport costs
transp_iron_ore_pellets

##########################################################################################################

# HBI transport

## transport costs
transp_HBI

##########################################################################################################

# H2 transport

## transport costs
transp_H2

##########################################################################################################

0

### Making a table of all economic results for each process step

In [52]:
labels = ['Electrolyzer',
          "Electrolyzer",
          "Electrolyzer",
          "Electrolyzer",
          "Electrolyzer",
          "Electrolyzer",
          'EAF',
          'EAF',
          'EAF',
          'EAF',
          'EAF',
          'EAF',
          'Iron ore pre-heat',
          'Iron ore pre-heat',
          'Iron ore pre-heat',
          'Iron ore pre-heat',
          'Iron ore pre-heat',
          'Iron ore pre-heat',
          'H2 pre-heat',
          'H2 pre-heat',
          'H2 pre-heat',
          'H2 pre-heat',
          'H2 pre-heat',
          'H2 pre-heat',
          "Shaft",
          "Shaft",
          "Shaft",
          "Shaft",
          "Shaft",
          "Shaft",
          "HBI pre-heat",
          "HBI pre-heat",
          "HBI pre-heat",
          "HBI pre-heat",
          "HBI pre-heat",
          "HBI pre-heat",
          "Iron ore transport",
          "HBI transport",
          "H2 transport"]
category=["CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "CAPEX",
          "Resource costs",
          "Electricity costs",
          "O&M costs",
          "Labour costs",
          "O2 revenues",
          "Transport costs",
          "Transport costs",
          "Transport costs" ]
len(category)

values=[ACC_el, 0, elec_electricity_cost, elec_OM_cost, labour_cost_elec, O2_rev,
       ACC_eaf, eaf_resource_costs, eaf_electricity_cost, eaf_OM_cost, labour_cost_eaf, 0,
       0, iron_ore_opex, iron_ore_preheat_electricity_cost, 0, 0, 0,
       0, 0, H2_preheat_electricity_cost, 0, 0, 0,
       ACC_shaft, 0, shaft_electricity_cost, shaft_OM_cost, labour_cost_shaft, 0,
       0, 0, HBI_preheat_electricity_cost, 0, 0, 0,
       transp_iron_ore_pellets,
       transp_HBI,
       transp_H2]

units=np.repeat("Euro/tLS",len(category))


lcos_breakdown={"Process step": labels,\
              "Category":category,\
              "Values": values,\
              "Units":units}

LCOS=pd.DataFrame(lcos_breakdown)
LCOS.index=LCOS['Process step']
LCOS

,Process step,Category,Values,Units
Process step,,,,
Electrolyzer,Electrolyzer,CAPEX,26.315489,Euro/tLS
Electrolyzer,Electrolyzer,Resource costs,0.000000,Euro/tLS
Electrolyzer,Electrolyzer,Electricity costs,118.871120,Euro/tLS
Electrolyzer,Electrolyzer,O&M costs,4.850919,Euro/tLS
Electrolyzer,Electrolyzer,Labour costs,2.657211,Euro/tLS
Electrolyzer,Electrolyzer,O2 revenues,-15.008914,Euro/tLS
EAF,EAF,CAPEX,21.612571,Euro/tLS
EAF,EAF,Resource costs,32.047000,Euro/tLS
EAF,EAF,Electricity costs,37.217689,Euro/tLS
